# Load gauge time series from netCDF file

This notebook provides a function to read in a netCDF file containing gauge time series output from the GeoClaw model at a set of gauges for each event in a set of events, e.g. the 36 ground motions developed for the Cascadia CoPes Hub.

This is a draft provided to solicit input on the format of these files.

A sample data file can be downloaded from
https://depts.washington.edu/ptha/CopesHubTsunamis/data/allgauges_Seaside_18buried.nc

This has data for the 18 "buried rupture" events at a set of 62 gauges in Seaside, OR, for a 90 minute tsunami simulation.

The gauge locations are shown in the map at the top of [this page](https://depts.washington.edu/ptha/CopesHubTsunamis/geoclaw_runs/sites/seaside/multirun2_hyak_2024-11-11/geoclaw_plots/), or you can download the [SeasideGauges.kml](https://depts.washington.edu/ptha/CopesHubTsunamis/geoclaw_runs/sites/seaside/multirun2_hyak_2024-11-11/geoclaw_plots/SeasideGauges.kml) and open it in Google Earth.

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import xarray

## Define a function for reading the netCDF file:

This function can be put in a module and imported in any code where you want to read in the gauge data.

In [ ]:
def read_allgauges_nc(ncfile):

    """
    Read all gauge data for a particular coastal site from a netCDF file
    that contains time series at all gauges pre-selected in this region,
    for some set of earthquake events.

    Time series for some quantities of interest (qoi) have been saved,
    typically ['h' 'u' 'v' 'eta'], where h is the water depth and
    eta is the surface elevation relative to the vertical datum of the
    topography file (e.g. MHW).

    :Input:  
        - ncfile : str
            path to the netCDF file

    :Output:
        - gauge_x : xarray.DataArray
            longitude of each gauge, indexed by gaugeno
        - gauge_y : xarray.DataArray
            latitude of each gauge, indexed by gaugeno
        - gauge_t : numpy.ndarray
            times for the time series, in seconds
            for convenience in plotting, with values from
            gauge_vals.coords.time
        - gauge_vals : xarray.DataArray
            4-dimensional array with all time series values
            
         
    """
    
    import xarray

    with xarray.open_dataset(ncfile, decode_timedelta=False) as ncdata:
        print(ncdata.description)
        gaugeno = ncdata.variables['gaugeno']
        x = ncdata.variables['x']
        y = ncdata.variables['y']
        time = ncdata.variables['time']
        event = ncdata.variables['event']
        qoi = ncdata.variables['qoi']
        gauge_vals = ncdata.variables['gauge_vals']

        # create xarrays to pass back:
        dims = ('time','gaugeno','qoi','event')
        coords = (time, gaugeno, qoi, event)
        gauge_vals = xarray.DataArray(gauge_vals, coords, dims)

        gauge_x = xarray.DataArray(x, (gaugeno,), ('gaugeno',))
        gauge_y = xarray.DataArray(y, (gaugeno,), ('gaugeno',))
        gauge_t = time.data  # simple numpy.ndarray for convenience

    print('Loaded %i times from t = %.1f to %.1f sec at %i gauges' \
            % (len(time),time[0],time[-1],len(x)))
    print('   for %i events, with qois %s' % (len(event), qoi.data))
    print('print gauge_vals.coords for more info')

    return gauge_x, gauge_y, gauge_t, gauge_vals
    

## Use this function to read in a file

This example uses the data file available at
https://depts.washington.edu/ptha/CopesHubTsunamis/data/allgauges_Seaside_random-str10.nc
To run this notebook, download this file and make sure `ncfile` includes the path to this file if it is not in the same directory as this notebook.

In [ ]:
ncfile = 'allgauges_Seaside_18buried.nc'
gauge_x, gauge_y, gauge_t, gauge_vals = read_allgauges_nc(ncfile)

### Examine the data:

In [ ]:
gauge_vals

To see the events included in this file you click on the disk icon next to `event` above, or print out the array of 'event' coordinates:

In [ ]:
gauge_vals.coords['event'].data

### index into numpy.ndarray

Note that  `gauge_vals.data` is an ordinary `numpy.ndarray`, but to index into this array you need to know how the dimensions or ordered, and what index corresponds to the desired gauge number, or quantity of interest, or event...

In [ ]:
gauge_vals.data.shape

### index into xarray:

It is easier to work with the `xarray.DataArray`, which provides a wrapper on top of the data to allow indexing directly by values of time, gauge number, event name, qoi name.

The `gauge_vals.sel` function allows specifying one or more dimensions.  If we specify 3 of the values as in the example below, it returns a 1-dimensional  `xarray.DataArray` where only `time` varies::

In [ ]:
mygaugeno = 1045
myevent = 'buried-random-str10-deep'

h = gauge_vals.sel(gaugeno=mygaugeno, qoi='h', event=myevent)
h

### Location of gauges:

`gauge_x` and `gauge_y` contain the longitude and latitude of each gauge: 

In [ ]:
gauge_x

In [ ]:
x = gauge_x.sel(gaugeno=mygaugeno)
y = gauge_y.sel(gaugeno=mygaugeno)
print('Gauge %i is at (%.5f, %.5f)' % (mygaugeno, x, y))

### Plot the water depth `h` at one gauge, for one event:

In [ ]:
figure(figsize=(10,6))
tminutes = gauge_t / 60.
plot(tminutes, h, 'b')
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,x,y,myevent));

### Compute and plot momentum flux at this gauge

In [ ]:
h = gauge_vals.sel(gaugeno=mygaugeno, qoi='h', event=myevent)
u = gauge_vals.sel(gaugeno=mygaugeno, qoi='u', event=myevent)
v = gauge_vals.sel(gaugeno=mygaugeno, qoi='v', event=myevent)
mflux = h * (u**2 + v**2)

figure(figsize=(10,6))
plot(gauge_t, mflux, 'b')
grid(True)
xlabel('minutes')
ylabel('m**3 / s**2')
title('Momentum flux at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,x,y,myevent));

### Select mulitple gauges for one event:

You can also select multiple gauges, in the case below `h` will be a 2-dimensional array indexed by `time` and `gaugeno`, but with only 4 gauges:

In [ ]:
gaugenos = range(1001,1005)
h = gauge_vals.sel(gaugeno=gaugenos, qoi='h', event=myevent)
h

In [ ]:
figure(figsize=(10,6))
for gaugeno in gaugenos: 
    plot(tminutes, h.sel(gaugeno=gaugeno), label='Gauge %i' % gaugeno)
legend()
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauges %s\nEvent %s' % (gaugenos,myevent));

### Select a subset of events at one gauge:

In [ ]:
events = ['buried-random-str10-deep', 'buried-random-str10-middle']
h = gauge_vals.sel(gaugeno=mygaugeno, event=events, qoi='h')
h

In [ ]:
figure(figsize=(10,6))
for ev in events:
    plot(tminutes, h.sel(event=ev), label=ev)
legend()
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %s' % mygaugeno);

### plot all the events at this gauge:

In [ ]:
events = gauge_vals.coords['event'].data  # all events
h = gauge_vals.sel(gaugeno=mygaugeno, event=events, qoi='h')

figure(figsize=(10,6))
for ev in events:
    plot(tminutes, h.sel(event=ev), label=ev)
legend()
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %s' % mygaugeno);

### What's the maximum value at this gauge over all events, and which event is largest?

From the plot above it's hard to tell which event corresponds to the upper-most curve, but we can identify this using `argmax`.

In [ ]:
float(h.max())

Which event has this maximum value?

In [ ]:
hmax = h.max(dim='time')
int(hmax.argmax(dim='event'))

`argmax` returned the integer input but we can see which event this is by indexing into the `events` array using this index:

In [ ]:
events[hmax.argmax(dim='event')]

### Plot this largest event to confirm:

In [ ]:
bigevent = events[hmax.argmax(dim='event')]
figure(figsize=(10,6))
tminutes = gauge_t / 60.
plot(tminutes, h.sel(event=bigevent), 'b')
grid(True)
xlabel('minutes')
ylabel('meters')
title('Water depth at Gauge %i (%.5f, %.5f)\nEvent %s' % (mygaugeno,x,y,bigevent));